# Preprocessing

## adding necessary tools

first we install dgl

In [1]:
#installing deep graph library
!pip3 install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.1 MB/s eta 0:00:00


now installing dglgo for some more functions

In [2]:
!pip install dglgo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 61.0 MB/s eta 0:00:00
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.wh

In this part we import and add some tools and modules

In [3]:
%matplotlib inline
import os #to create and dealing with directory

os.environ["DGLBACKEND"] = "pytorch"
import dgl #deep graph library
import numpy as np #numerical functions
import networkx as nx #to represent graph and dealing with graph
import torch #for neural network
import torch.nn as nn
import dgl.function as fn #to catch functions
import torch.nn.functional as F #neural network functions
import shutil #help us import dataset
from torch.utils.data import DataLoader #to load data
import cloudpickle
from dgl.nn import GraphConv
from sklearn import preprocessing

## loading dataset

using HIV dataset<br>
HIV dataset was introduced by the Drug Therapeutics Program (DTP) AIDS Antiviral Screen, which tested the ability to inhibit HIV replication for over 40000 compounds.

By using shutil we can bring dataset from origin location

In [4]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "graph_classification.zip"
shutil.unpack_archive(path_save, folder_data_temp)

Defining a custom pytorch classification dataset class

In [5]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
            self.address=address+".bin"
            self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
            num_graphs =len(self.list_graphs)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
    def __len__(self):
        return len(self.list_graphs)
    def __getitem__(self, idx):
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]


performing train and test and split according to file<br>
we bring data from its origin location. at origin location there were 3 scaffold which represent on scaffold in dataset so we choose first one of scaffold0 and then by DGLDataset we bring train and test and validaion.

In [6]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0) #str(0) is because we use scaffold_0 from main datasets at source
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))

32901 4112 4114


At this part we catch graphs, labels, masks, globals from batches tuple.


In [7]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals



then we define a loader with batch size of 64 since dataset is a bit big.
we'll define 3 dataloaders for train and validation and test, batch size for each is 64 and collate_fn is according to collate we defined earlier before defining loader

In [8]:
def loader(batch_size = 64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [9]:
#defining new train test validation by dataloader function
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

# Model Defining

Now it's time to define model,
number of tasks in this dataset is 1 and we set number of epochs to 100 which looks enough for this kind of task. global size is 200 which represents global feature of each graph and patience is 10 which is Number of steps to wait if the model performance on the validation set does not improve.
for configuraton it's better to set node_feature_size to 127, edge_feature_size to 12 and hidden size to 100 in order to avoid further problems.

In [10]:
#HIV has only 1 task
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}

## defining graph sage

In [11]:
#defining Graph sage class from scratch
class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat): #in_feat is input feature size in integer and out_feat is output feature size in integer
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h): # g is input graph and h is input node feature
        with g.local_scope(): #local_scope is to stop out-place mutation to the feature data not reflecting original graph
            g.ndata["h"] = h
            # message passing step
            g.update_all(
                message_func=fn.copy_u("h", "m"), #perorming message function by copy_u function from dgl
                reduce_func=fn.mean("m", "h_N"), #performing reduce function by mean
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1) #concating
            return self.linear(h_total)

for this project we'll use 20 models

## defining models

model1<br>
using graph convolutional<br>
2 layer<br>
no drop out

In [12]:
class model1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True) #allow_zero_in_degree is dou to avoid further errors because of 0 degree nodes(isolated nodes)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]) #first layer
        h = F.relu(h) #relu as activaton function
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model2<br>
using graph sage<br>
2 layer<br>
no drop out

In [13]:
class model2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]) #first layer
        h = F.relu(h) #relu as activaton function
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model3<br>
using graph convolutional<br>
3 layer<br>
no drop out

In [14]:
class model3(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model4<br>
using graph sage<br>
3 layer<br>
no drop out

In [15]:
class model4(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model5<br>
using graph convolutional<br>
4 layer<br>
no drop out

In [16]:
class model5(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model6<br>
using graph convolutional<br>
4 layer<br>
dropout = 0.1

In [17]:
class model6(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model7<br>
using graph convolutional<br>
4 layer<br>
dropout = 0.2

In [18]:
class model7(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model8<br>
using graph convolutional<br>
4 layer<br>
dropout = 0.25

In [19]:
class model8(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model9<br>
using graph sage<br>
4 layer<br>
dropout = 0

In [20]:
class model9(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model10<br>
using graph sage<br>
4 layer<br>
dropout = 0.1

In [21]:
class model10(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model11<br>
using graph sage<br>
4 layer<br>
dropout = 0.2

In [22]:
class model11(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model12<br>
using graph sage<br>
4 layer<br>
dropout = 0.25

In [23]:
class model12(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model13<br>
using graph convolutional<br>
5 layer<br>
dropout = 0

In [24]:
class model13(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model14<br>
using graph convolutional<br>
5 layer<br>
dropout = 0.1

In [25]:
class model14(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model15<br>
using graph convolutional<br>
5 layer<br>
dropout = 0.2

In [26]:
class model15(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model16<br>
using graph convolutional<br>
5 layer<br>
dropout = 0.25

In [27]:
class model16(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model17<br>
using graph sage<br>
5 layer<br>
dropout = 0

In [28]:
class model17(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model18<br>
using graph sage<br>
5 layer<br>
dropout = 0.1

In [29]:
class model18(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model19<br>
using graph sage<br>
5 layer<br>
dropout = 0.2

In [30]:
class model19(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model20<br>
using graph sage<br>
5 layer<br>
dropout = 0.25

In [31]:
class model20(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

Function to Compute Score of the Model

In [32]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

loss function

In [33]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

training function

In [34]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

## model1

using graph convolutional<br>
3 layer<br>
no drop out

In [35]:
class model1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

Function to Compute Score of the Model

In [36]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

loss function

In [37]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

training function

In [38]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [39]:
def train_evaluate():

    model = model1(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

Function to compute test set score of the final saved model

In [40]:
def test_evaluate():
    final_model = model1(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

Train the model and evaluate its performance

In [41]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.202 | Valid Score: 0.554
 
Epoch: 1/100 | Best Valid Score Until Now: 0.554 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.159 | Valid Score: 0.636
 
Epoch: 2/100 | Best Valid Score Until Now: 0.636 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.152 | Valid Score: 0.695
 
Epoch: 3/100 | Best Valid Score Until Now: 0.695 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.148 | Valid Score: 0.727
 
Epoch: 4/100 | Best Valid Score Until Now: 0.727 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.147 | Valid Score: 0.744
 
Epoch: 5/100 | Best Valid Score Until Now: 0.744 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.145 | Valid Score: 0.752
 
Epoch: 6/100 | Best Valid Score Until Now: 0.752 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.145 | Valid Score: 0.757
 
Epoch: 7/100 | Best Valid Score Until Now: 0.757 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.144 | Valid Score: 0.759
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 2

In [42]:
def train_evaluate():

    model = model2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [43]:
def test_evaluate():
    final_model = model2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [44]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.256 | Valid Score: 0.531
 
Epoch: 1/100 | Best Valid Score Until Now: 0.531 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.165 | Valid Score: 0.544
 
Epoch: 2/100 | Best Valid Score Until Now: 0.544 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.161 | Valid Score: 0.572
 
Epoch: 3/100 | Best Valid Score Until Now: 0.572 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.156 | Valid Score: 0.613
 
Epoch: 4/100 | Best Valid Score Until Now: 0.613 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.152 | Valid Score: 0.680
 
Epoch: 5/100 | Best Valid Score Until Now: 0.680 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.148 | Valid Score: 0.704
 
Epoch: 6/100 | Best Valid Score Until Now: 0.704 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.148 | Valid Score: 0.721
 
Epoch: 7/100 | Best Valid Score Until Now: 0.721 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.146 | Valid Score: 0.731
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 3

In [45]:
def train_evaluate():

    model = model3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [46]:
def test_evaluate():
    final_model = model3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [47]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.190 | Valid Score: 0.575
 
Epoch: 1/100 | Best Valid Score Until Now: 0.575 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.158 | Valid Score: 0.644
 
Epoch: 2/100 | Best Valid Score Until Now: 0.644 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.151 | Valid Score: 0.696
 
Epoch: 3/100 | Best Valid Score Until Now: 0.696 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.148 | Valid Score: 0.731
 
Epoch: 4/100 | Best Valid Score Until Now: 0.731 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.146 | Valid Score: 0.744
 
Epoch: 5/100 | Best Valid Score Until Now: 0.744 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.145 | Valid Score: 0.751
 
Epoch: 6/100 | Best Valid Score Until Now: 0.751 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.145 | Valid Score: 0.757
 
Epoch: 7/100 | Best Valid Score Until Now: 0.757 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.145 | Valid Score: 0.759
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 4

In [48]:
def train_evaluate():

    model = model4(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [49]:
def test_evaluate():
    final_model = model4(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [50]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.227 | Valid Score: 0.561
 
Epoch: 1/100 | Best Valid Score Until Now: 0.561 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.156 | Valid Score: 0.659
 
Epoch: 2/100 | Best Valid Score Until Now: 0.659 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.149 | Valid Score: 0.727
 
Epoch: 3/100 | Best Valid Score Until Now: 0.727 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.146 | Valid Score: 0.741
 
Epoch: 4/100 | Best Valid Score Until Now: 0.741 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.144 | Valid Score: 0.746
 
Epoch: 5/100 | Best Valid Score Until Now: 0.746 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.143 | Valid Score: 0.747
 
Epoch: 6/100 | Best Valid Score Until Now: 0.747 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.144 | Valid Score: 0.749
 
Epoch: 7/100 | Best Valid Score Until Now: 0.749 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.141 | Valid Score: 0.753
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 5

In [51]:
def train_evaluate():

    model = model5(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [52]:
def test_evaluate():
    final_model = model5(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [53]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.218 | Valid Score: 0.543
 
Epoch: 1/100 | Best Valid Score Until Now: 0.543 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.155 | Valid Score: 0.674
 
Epoch: 2/100 | Best Valid Score Until Now: 0.674 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.148 | Valid Score: 0.721
 
Epoch: 3/100 | Best Valid Score Until Now: 0.721 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.145 | Valid Score: 0.745
 
Epoch: 4/100 | Best Valid Score Until Now: 0.745 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.144 | Valid Score: 0.754
 
Epoch: 5/100 | Best Valid Score Until Now: 0.754 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.143 | Valid Score: 0.756
 
Epoch: 6/100 | Best Valid Score Until Now: 0.756 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.142 | Valid Score: 0.757
 
Epoch: 7/100 | Best Valid Score Until Now: 0.757 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.141 | Valid Score: 0.762
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 6

In [54]:
def train_evaluate():

    model = model6(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [55]:
def test_evaluate():
    final_model = model6(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [56]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.201 | Valid Score: 0.574
 
Epoch: 1/100 | Best Valid Score Until Now: 0.574 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.154 | Valid Score: 0.691
 
Epoch: 2/100 | Best Valid Score Until Now: 0.691 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.147 | Valid Score: 0.735
 
Epoch: 3/100 | Best Valid Score Until Now: 0.735 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.144 | Valid Score: 0.746
 
Epoch: 4/100 | Best Valid Score Until Now: 0.746 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.751
 
Epoch: 5/100 | Best Valid Score Until Now: 0.751 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.142 | Valid Score: 0.757
 
Epoch: 6/100 | Best Valid Score Until Now: 0.757 

Patience 1
Epoch: 7/100 | Training Loss: 0.141 | Valid Score: 0.754
 
Epoch: 7/100 | Best Valid Score Until Now: 0.757 

Patience 2
Epoch: 8/100 | Training Loss: 0.140 | Valid Score: 0.755
 
Epoch: 8/100 | Best Valid Score Until Now: 0.757 

Sa

model 7

In [57]:
def train_evaluate():

    model = model7(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [58]:
def test_evaluate():
    final_model = model7(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [59]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.226 | Valid Score: 0.551
 
Epoch: 1/100 | Best Valid Score Until Now: 0.551 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.156 | Valid Score: 0.652
 
Epoch: 2/100 | Best Valid Score Until Now: 0.652 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.150 | Valid Score: 0.723
 
Epoch: 3/100 | Best Valid Score Until Now: 0.723 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.146 | Valid Score: 0.747
 
Epoch: 4/100 | Best Valid Score Until Now: 0.747 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.752
 
Epoch: 5/100 | Best Valid Score Until Now: 0.752 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.142 | Valid Score: 0.754
 
Epoch: 6/100 | Best Valid Score Until Now: 0.754 

Patience 1
Epoch: 7/100 | Training Loss: 0.141 | Valid Score: 0.752
 
Epoch: 7/100 | Best Valid Score Until Now: 0.754 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.141 | Valid Score: 0.756
 
Epoch: 8/100 | Best Valid Score Until Now: 0.756

model 8

In [60]:
def train_evaluate():

    model = model8(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [61]:
def test_evaluate():
    final_model = model8(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [62]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.219 | Valid Score: 0.551
 
Epoch: 1/100 | Best Valid Score Until Now: 0.551 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.156 | Valid Score: 0.664
 
Epoch: 2/100 | Best Valid Score Until Now: 0.664 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.149 | Valid Score: 0.726
 
Epoch: 3/100 | Best Valid Score Until Now: 0.726 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.146 | Valid Score: 0.744
 
Epoch: 4/100 | Best Valid Score Until Now: 0.744 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.744
 
Epoch: 5/100 | Best Valid Score Until Now: 0.744 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.143 | Valid Score: 0.751
 
Epoch: 6/100 | Best Valid Score Until Now: 0.751 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.143 | Valid Score: 0.754
 
Epoch: 7/100 | Best Valid Score Until Now: 0.754 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.143 | Valid Score: 0.756
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 9

In [63]:
def train_evaluate():

    model = model9(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [64]:
def test_evaluate():
    final_model = model9(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [65]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.212 | Valid Score: 0.571
 
Epoch: 1/100 | Best Valid Score Until Now: 0.571 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.152 | Valid Score: 0.713
 
Epoch: 2/100 | Best Valid Score Until Now: 0.713 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.146 | Valid Score: 0.737
 
Epoch: 3/100 | Best Valid Score Until Now: 0.737 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.143 | Valid Score: 0.742
 
Epoch: 4/100 | Best Valid Score Until Now: 0.742 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.142 | Valid Score: 0.747
 
Epoch: 5/100 | Best Valid Score Until Now: 0.747 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.141 | Valid Score: 0.758
 
Epoch: 6/100 | Best Valid Score Until Now: 0.758 

Patience 1
Epoch: 7/100 | Training Loss: 0.140 | Valid Score: 0.757
 
Epoch: 7/100 | Best Valid Score Until Now: 0.758 

Patience 2
Epoch: 8/100 | Training Loss: 0.140 | Valid Score: 0.758
 
Epoch: 8/100 | Best Valid Score Until Now: 0.758 

Sa

model 10

In [66]:
def train_evaluate():

    model = model10(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [67]:
def test_evaluate():
    final_model = model10(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [68]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.226 | Valid Score: 0.616
 
Epoch: 1/100 | Best Valid Score Until Now: 0.616 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.152 | Valid Score: 0.721
 
Epoch: 2/100 | Best Valid Score Until Now: 0.721 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.146 | Valid Score: 0.750
 
Epoch: 3/100 | Best Valid Score Until Now: 0.750 

Patience 1
Epoch: 4/100 | Training Loss: 0.144 | Valid Score: 0.748
 
Epoch: 4/100 | Best Valid Score Until Now: 0.750 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.755
 
Epoch: 5/100 | Best Valid Score Until Now: 0.755 

Patience 1
Epoch: 6/100 | Training Loss: 0.142 | Valid Score: 0.747
 
Epoch: 6/100 | Best Valid Score Until Now: 0.755 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.140 | Valid Score: 0.762
 
Epoch: 7/100 | Best Valid Score Until Now: 0.762 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.139 | Valid Score: 0.766
 
Epoch: 8/100 | Best Valid Score Until Now: 0.766 

Sa

model 11

In [69]:
def train_evaluate():

    model = model11(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [70]:
def test_evaluate():
    final_model = model11(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [71]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.217 | Valid Score: 0.592
 
Epoch: 1/100 | Best Valid Score Until Now: 0.592 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.151 | Valid Score: 0.731
 
Epoch: 2/100 | Best Valid Score Until Now: 0.731 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.146 | Valid Score: 0.750
 
Epoch: 3/100 | Best Valid Score Until Now: 0.750 

Patience 1
Epoch: 4/100 | Training Loss: 0.143 | Valid Score: 0.746
 
Epoch: 4/100 | Best Valid Score Until Now: 0.750 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.145 | Valid Score: 0.751
 
Epoch: 5/100 | Best Valid Score Until Now: 0.751 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.142 | Valid Score: 0.755
 
Epoch: 6/100 | Best Valid Score Until Now: 0.755 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.140 | Valid Score: 0.760
 
Epoch: 7/100 | Best Valid Score Until Now: 0.760 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.139 | Valid Score: 0.763
 
Epoch: 8/100 | Best Valid Score Until Now: 0.763

model 12

In [72]:
def train_evaluate():

    model = model12(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [73]:
def test_evaluate():
    final_model = model12(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [74]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.219 | Valid Score: 0.599
 
Epoch: 1/100 | Best Valid Score Until Now: 0.599 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.151 | Valid Score: 0.730
 
Epoch: 2/100 | Best Valid Score Until Now: 0.730 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.146 | Valid Score: 0.751
 
Epoch: 3/100 | Best Valid Score Until Now: 0.751 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.144 | Valid Score: 0.754
 
Epoch: 4/100 | Best Valid Score Until Now: 0.754 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.142 | Valid Score: 0.759
 
Epoch: 5/100 | Best Valid Score Until Now: 0.759 

Patience 1
Epoch: 6/100 | Training Loss: 0.142 | Valid Score: 0.757
 
Epoch: 6/100 | Best Valid Score Until Now: 0.759 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.141 | Valid Score: 0.763
 
Epoch: 7/100 | Best Valid Score Until Now: 0.763 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.140 | Valid Score: 0.765
 
Epoch: 8/100 | Best Valid Score Until Now: 0.765

model 13

In [75]:
def train_evaluate():

    model = model13(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [76]:
def test_evaluate():
    final_model = model13(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [77]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.225 | Valid Score: 0.620
 
Epoch: 1/100 | Best Valid Score Until Now: 0.620 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.152 | Valid Score: 0.710
 
Epoch: 2/100 | Best Valid Score Until Now: 0.710 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.147 | Valid Score: 0.752
 
Epoch: 3/100 | Best Valid Score Until Now: 0.752 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.145 | Valid Score: 0.763
 
Epoch: 4/100 | Best Valid Score Until Now: 0.763 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.764
 
Epoch: 5/100 | Best Valid Score Until Now: 0.764 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.143 | Valid Score: 0.767
 
Epoch: 6/100 | Best Valid Score Until Now: 0.767 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.142 | Valid Score: 0.768
 
Epoch: 7/100 | Best Valid Score Until Now: 0.768 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.141 | Valid Score: 0.771
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 14

In [78]:
def train_evaluate():

    model = model14(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [79]:
def test_evaluate():
    final_model = model14(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [80]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.214 | Valid Score: 0.614
 
Epoch: 1/100 | Best Valid Score Until Now: 0.614 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.154 | Valid Score: 0.706
 
Epoch: 2/100 | Best Valid Score Until Now: 0.706 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.149 | Valid Score: 0.741
 
Epoch: 3/100 | Best Valid Score Until Now: 0.741 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.148 | Valid Score: 0.757
 
Epoch: 4/100 | Best Valid Score Until Now: 0.757 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.146 | Valid Score: 0.763
 
Epoch: 5/100 | Best Valid Score Until Now: 0.763 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.145 | Valid Score: 0.765
 
Epoch: 6/100 | Best Valid Score Until Now: 0.765 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.143 | Valid Score: 0.768
 
Epoch: 7/100 | Best Valid Score Until Now: 0.768 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.142 | Valid Score: 0.772
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 15

In [81]:
def train_evaluate():

    model = model15(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [82]:
def test_evaluate():
    final_model = model15(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [83]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.223 | Valid Score: 0.589
 
Epoch: 1/100 | Best Valid Score Until Now: 0.589 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.155 | Valid Score: 0.696
 
Epoch: 2/100 | Best Valid Score Until Now: 0.696 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.150 | Valid Score: 0.754
 
Epoch: 3/100 | Best Valid Score Until Now: 0.754 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.147 | Valid Score: 0.759
 
Epoch: 4/100 | Best Valid Score Until Now: 0.759 

Patience 1
Epoch: 5/100 | Training Loss: 0.144 | Valid Score: 0.757
 
Epoch: 5/100 | Best Valid Score Until Now: 0.759 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.143 | Valid Score: 0.760
 
Epoch: 6/100 | Best Valid Score Until Now: 0.760 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.142 | Valid Score: 0.764
 
Epoch: 7/100 | Best Valid Score Until Now: 0.764 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.142 | Valid Score: 0.765
 
Epoch: 8/100 | Best Valid Score Until Now: 0.765

model 16

In [84]:
def train_evaluate():

    model = model16(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [85]:
def test_evaluate():
    final_model = model16(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [86]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.205 | Valid Score: 0.597
 
Epoch: 1/100 | Best Valid Score Until Now: 0.597 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.154 | Valid Score: 0.699
 
Epoch: 2/100 | Best Valid Score Until Now: 0.699 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.149 | Valid Score: 0.731
 
Epoch: 3/100 | Best Valid Score Until Now: 0.731 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.148 | Valid Score: 0.744
 
Epoch: 4/100 | Best Valid Score Until Now: 0.744 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.146 | Valid Score: 0.749
 
Epoch: 5/100 | Best Valid Score Until Now: 0.749 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.145 | Valid Score: 0.755
 
Epoch: 6/100 | Best Valid Score Until Now: 0.755 

Patience 1
Epoch: 7/100 | Training Loss: 0.144 | Valid Score: 0.751
 
Epoch: 7/100 | Best Valid Score Until Now: 0.755 

Patience 2
Epoch: 8/100 | Training Loss: 0.144 | Valid Score: 0.754
 
Epoch: 8/100 | Best Valid Score Until Now: 0.755 

Sa

model 17

In [87]:
def train_evaluate():

    model = model17(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [88]:
def test_evaluate():
    final_model = model17(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [89]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.209 | Valid Score: 0.654
 
Epoch: 1/100 | Best Valid Score Until Now: 0.654 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.148 | Valid Score: 0.742
 
Epoch: 2/100 | Best Valid Score Until Now: 0.742 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.146 | Valid Score: 0.752
 
Epoch: 3/100 | Best Valid Score Until Now: 0.752 

Patience 1
Epoch: 4/100 | Training Loss: 0.142 | Valid Score: 0.744
 
Epoch: 4/100 | Best Valid Score Until Now: 0.752 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.142 | Valid Score: 0.755
 
Epoch: 5/100 | Best Valid Score Until Now: 0.755 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.140 | Valid Score: 0.758
 
Epoch: 6/100 | Best Valid Score Until Now: 0.758 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.139 | Valid Score: 0.770
 
Epoch: 7/100 | Best Valid Score Until Now: 0.770 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.137 | Valid Score: 0.770
 
Epoch: 8/100 | Best Valid Score Until Now: 0.770

model 18

In [90]:
def train_evaluate():

    model = model18(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [91]:
def test_evaluate():
    final_model = model18(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [92]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.221 | Valid Score: 0.641
 
Epoch: 1/100 | Best Valid Score Until Now: 0.641 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.149 | Valid Score: 0.735
 
Epoch: 2/100 | Best Valid Score Until Now: 0.735 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.144 | Valid Score: 0.757
 
Epoch: 3/100 | Best Valid Score Until Now: 0.757 

Patience 1
Epoch: 4/100 | Training Loss: 0.143 | Valid Score: 0.748
 
Epoch: 4/100 | Best Valid Score Until Now: 0.757 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.142 | Valid Score: 0.761
 
Epoch: 5/100 | Best Valid Score Until Now: 0.761 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.140 | Valid Score: 0.770
 
Epoch: 6/100 | Best Valid Score Until Now: 0.770 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.139 | Valid Score: 0.773
 
Epoch: 7/100 | Best Valid Score Until Now: 0.773 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.137 | Valid Score: 0.776
 
Epoch: 8/100 | Best Valid Score Until Now: 0.776

model 19

In [93]:
def train_evaluate():

    model = model19(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [94]:
def test_evaluate():
    final_model = model19(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [95]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.213 | Valid Score: 0.631
 
Epoch: 1/100 | Best Valid Score Until Now: 0.631 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.149 | Valid Score: 0.743
 
Epoch: 2/100 | Best Valid Score Until Now: 0.743 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.145 | Valid Score: 0.753
 
Epoch: 3/100 | Best Valid Score Until Now: 0.753 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.142 | Valid Score: 0.760
 
Epoch: 4/100 | Best Valid Score Until Now: 0.760 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.141 | Valid Score: 0.763
 
Epoch: 5/100 | Best Valid Score Until Now: 0.763 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.139 | Valid Score: 0.764
 
Epoch: 6/100 | Best Valid Score Until Now: 0.764 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.138 | Valid Score: 0.771
 
Epoch: 7/100 | Best Valid Score Until Now: 0.771 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.137 | Valid Score: 0.777
 
Epoch: 8/100 | Best Valid Score Until Now: 

model 20

In [96]:
def train_evaluate():

    model = model20(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [97]:
def test_evaluate():
    final_model = model20(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [98]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.214 | Valid Score: 0.606
 
Epoch: 1/100 | Best Valid Score Until Now: 0.606 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.152 | Valid Score: 0.731
 
Epoch: 2/100 | Best Valid Score Until Now: 0.731 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.146 | Valid Score: 0.745
 
Epoch: 3/100 | Best Valid Score Until Now: 0.745 

Patience 1
Epoch: 4/100 | Training Loss: 0.143 | Valid Score: 0.743
 
Epoch: 4/100 | Best Valid Score Until Now: 0.745 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.759
 
Epoch: 5/100 | Best Valid Score Until Now: 0.759 

Patience 1
Epoch: 6/100 | Training Loss: 0.141 | Valid Score: 0.748
 
Epoch: 6/100 | Best Valid Score Until Now: 0.759 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.141 | Valid Score: 0.766
 
Epoch: 7/100 | Best Valid Score Until Now: 0.766 

Patience 1
Epoch: 8/100 | Training Loss: 0.139 | Valid Score: 0.765
 
Epoch: 8/100 | Best Valid Score Until Now: 0.766 

Save ch